In [43]:
import os
import re
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [44]:
#user setup area
highestDWT = 1000000 #change this for the cap
vessel_type = 401 #401 bulk carrier , 402 general cargo , 403 container , 6 all tanker 
minYear = '1985'
maxYear = '2022'
flag = 'ID' #refer to vessel finder names

In [50]:
filename = 'bulkcarrier.csv'
directory ='C:/Users/abinp/Documents/ITS/side projects/new/imo scrapping/output/names/'
csv_write = '{}{}'.format(directory,filename)

In [35]:
%ls
link =''
vessel_names = []
minDW = -1000
maxDW = 0

 Volume in drive C has no label.
 Volume Serial Number is A071-85E6

 Directory of C:\Users\abinp\Documents\ITS\side projects\new\imo scrapping\scripts

21/04/2023  10:27    <DIR>          .
20/04/2023  20:05    <DIR>          ..
21/04/2023  10:27    <DIR>          .ipynb_checkpoints
20/04/2023  21:48             2.806 csv splitter.ipynb
20/04/2023  20:07            18.779 Imo kapal.ipynb
21/04/2023  10:27            15.789 Nama.ipynb
21/04/2023  10:27           483.404 Nama_ngapal 4.ipynb
               4 File(s)        520.778 bytes
               3 Dir(s)  22.654.926.848 bytes free


In [36]:
def changelink(min,max):
    global minDW , maxDW , link
    minDW = min + 1000
    maxDW = max + 1000000
    #https://www.vesselfinder.com/vessels?type=401&flag=ID&minDW=1000&maxDW=100000&minYear=1985&maxYear=2022
    link = "https://www.vesselfinder.com/vessels?type={}&flag={}&minDW={}&maxDW={}&minYear={}&maxYear={}&dir=1&sort=4".format(vessel_type,flag,minDW,maxDW,minYear,maxYear)   
changelink(minDW,maxDW)
print(link)

https://www.vesselfinder.com/vessels?type=401&flag=ID&minDW=0&maxDW=1000000&minYear=1985&maxYear=2022&dir=1&sort=4


In [37]:
# create webdriver instance using options
driver = webdriver.Chrome(executable_path=r"chromedriver")
driver.get(link)

In [38]:
while True:
    names = driver.find_elements(By.CLASS_NAME, "slna")
    for name in names:
        vessel_names.append(name.text)
    try:
        next_page = driver.find_element(By.CLASS_NAME, "pagination-next")
        if not next_page.is_enabled():
            print("Out of pages")
            break
        next_page.click()
        
    except NoSuchElementException:
        if maxDW == highestDWT:
            print("reaching end of range")
            break
        else:
            print("changing range |  min dw :{}   max dw: {}".format(minDW,maxDW))
            changelink(minDW,maxDW)
            driver = webdriver.Chrome(executable_path=r"chromedriver")
            driver.get(link)
            continue

reaching end of range


In [46]:
print(vessel_names)

['HABCO PIONEER', 'PUSRI INDONESIA 1', 'LUMOSO PROSPER', 'MV NAZIHA', 'KARUNIA', 'DK 02', 'DHARMA LAUTAN RUBY', 'CAHAYA MUSTIKA LAUT1', 'MBS BALURAN', 'AMANAH SULAWESI AMC', 'SINAR KINTAMANI', 'MANALAGI TISYA', 'ARFIANIE AYU', 'GOLDEN REJEKI', 'ASIAN WISDOM', 'KT 06', 'LUMOSO LESTARI', 'LUMOSO KARUNIA II', 'GUNALEILA', 'DEWI AMBARWATI', 'MUMTAZ', 'LUMOSO LANCAR', 'GOLDEN ROSE', 'MUNQIDZ', 'MANALAGI WANDA', 'MV PUTERI SEJATI', 'NOAH ASYERA', 'CHLOE', 'SAMUDRA SAKTI III', 'KT 02', 'MV. LUMOSO RAYA', 'FIRST KASIH', 'MV.MUHASYIR', 'KT 05', 'UNITAMA LILY', 'LUMOSO SURYA', '-SARTIKA BARUNA', 'LGH PROSPER', 'MANALAGI HITA', 'PANCARAN I 5505', 'KAMADIYA', 'PRIMA ANDALAN I', 'ANDHIKA ALISHA', 'DRY TRANSPORT', 'KM PUTERI KIRANA', 'LUMOSO KARUNIA VIII', 'ALIYAH PRATAMA', 'PERMATA CAROLINE', 'AMANAH MOROWALI AMC', 'MV.ARIMBI BARUNA', 'SRIKANDI INDONESIA19', 'MARTHA BARUNA', 'LUMOSO HAWARI', 'MANALAGI TARA', 'INDRANI', 'ANDHIKA PARAMESTI', 'ISA EXPRESS', 'LUMOSO AMAN', 'MV.ISA GOLDEN', 'LUMOSO KASI

In [47]:
duplicates = []
for name in vessel_names:
    if vessel_names.count(name) > 1:
        duplicates.append(name)

duplicates = list(set(duplicates))

for name in duplicates:
    print("Removing duplicates:", name)
    
#clear duplicate
clear_list = list(set(vessel_names))
vessel_names = clear_list

In [48]:
len(vessel_names)

161

In [51]:
# assume the list of names is stored in a variable called "vessel_names"

# open a new CSV file for writing
with open(csv_write, 'w', newline='') as csvfile:
    # create a CSV writer object
    writer = csv.writer(csvfile)
    
    # write the header row (optional)
    writer.writerow(['name'])

    # write each name and its corresponding minDW and maxDW values to a new row in the CSV file
    for i in range(0, len(vessel_names)):
        name = vessel_names[i]
        writer.writerow([name])
